In [9]:
!pip install python-dotenv

In [10]:
import os
import json

from dotenv import load_dotenv
from minsearch import Index

load_dotenv()
os.environ["HF_HOME"] = "/run/cache/"

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

print(input_ids)

tensor([[13959,  1566,    12,  2968,    10,   571,   625,    33,    25,    58,
             1]], device='cuda:0')


In [4]:
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alt sind Sie?</s>


In [5]:
print(outputs)

tensor([[   0, 2739, 4445,  436,  292,   58,    1]], device='cuda:0')


In [11]:


documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    

In [12]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [13]:
textfields = ["text", "section", "question"]


indobject = Index(text_fields = textfields, keyword_fields = ['course'])
indobject.fit(documents)

In [20]:


def search(query, boost_dict = {"question": 3}, filter_dict =  {"course":"mlops-zoomcamp"}, num_res = 5 ):

    
    context = indobject.search(query = query, boost_dict = boost_dict, filter_dict = filter_dict,
                               num_results = num_res
                    )
    
    return context

In [15]:
def build_prompt(query, related_docs):
    
    
    
    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [30]:
def chat(prompt,generate_params = {}):
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    
    outputs = model.generate(input_ids,
                            max_length = generate_params.get("max_length", 150),
                             
                            top_k = generate_params.get("top_k", 50),
                             
                            top_p = generate_params.get("top_p", 0.95),
                             
                            do_sample = generate_params.get("do_sample",False),
                             
                            num_beams = generate_params.get("num_beams", 5),
                             
                            temperature = generate_params.get("temperature", 1.0))
    
    answer = tokenizer.decode(outputs[0])
    return answer
    
    

In [27]:
def rag(query, boost_dict = {"question": 3}, course_filter =  {"course": "data-engineering-zoomcamp"}, generate_params ={}):
    
    context = search(query = query, boost_dict = boost_dict, filter_dict = course_filter)
    
    prompt = build_prompt(query, context)
    
    answer = chat(prompt, generate_params = generate_params)
    
    return answer
    

In [31]:
query = "I just found that course has already started can i still enroll in it?"
answer = rag(query = query)

In [32]:
answer

"<pad> Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.</s>"